In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import MulensModel as mm
import emcee
import corner
import os
from scipy.optimize import minimize
import sys

sys.path.append('../Modules')
from Guide import define, list_entries, list_notebooks, open_notebook